In [51]:
import pandas as pd
import numpy as np
import re
import json
import ast
import tweepy
from textblob import TextBlob
import os
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
import emoji
import yake

In [52]:
activity_watch = pd.read_json("../data/march_2022/activity_watch_march_2022_latest.json")
#separating the buckets column as that is where the dictionary of relevant data is

activity_watch = activity_watch['buckets'].apply(pd.Series)
activity_watch
#Further splitting the events column to extract the time stamp, duration and data


,id,created,name,type,client,hostname,events
aw-stopwatch,aw-stopwatch,2022-01-06T22:11:17.199172+00:00,None,general.stopwatch,aw-webui,unknown,[]
aw-watcher-afk_MacBook-Pro.hitronhub.home,aw-watcher-afk_MacBook-Pro.hitronhub.home,2022-01-04T21:15:38.304699+00:00,None,afkstatus,aw-watcher-afk,MacBook-Pro.hitronhub.home,[{'timestamp': '2022-01-21T02:49:18.638000+00:...
aw-watcher-afk_Ores-MBP.hitronhub.home,aw-watcher-afk_Ores-MBP.hitronhub.home,2022-01-20T18:59:44.917818+00:00,None,afkstatus,aw-watcher-afk,Ores-MBP.hitronhub.home,[{'timestamp': '2022-03-09T07:06:45.826000+00:...
aw-watcher-afk_Ores-MacBook-Pro.local,aw-watcher-afk_Ores-MacBook-Pro.local,2022-03-12T21:46:33.371075+00:00,None,afkstatus,aw-watcher-afk,Ores-MacBook-Pro.local,[{'timestamp': '2022-03-17T17:57:06.954000+00:...
aw-watcher-afk_SAW-L5487407OT9.hitronhub.home,aw-watcher-afk_SAW-L5487407OT9.hitronhub.home,2022-03-08T23:10:08.440588+00:00,None,afkstatus,aw-watcher-afk,SAW-L5487407OT9.hitronhub.home,[{'timestamp': '2022-03-13T05:41:19.815000+00:...
aw-watcher-afk_og-mbpro,aw-watcher-afk_og-mbpro,2022-03-17T11:26:13.407249+00:00,None,afkstatus,aw-watcher-afk,og-mbpro,[{'timestamp': '2022-03-22T12:55:13.330000+00:...
aw-watcher-window_MacBook-Pro.hitronhub.home,aw-watcher-window_MacBook-Pro.hitronhub.home,2022-01-04T21:15:37.737954+00:00,None,currentwindow,aw-watcher-window,MacBook-Pro.hitronhub.home,[{'timestamp': '2022-01-21T02:57:33.262000+00:...
aw-watcher-window_Ores-MBP.hitronhub.home,aw-watcher-window_Ores-MBP.hitronhub.home,2022-01-20T18:59:44.736005+00:00,None,currentwindow,aw-watcher-window,Ores-MBP.hitronhub.home,[{'timestamp': '2022-03-09T07:07:15.992000+00:...
aw-watcher-window_Ores-MacBook-Pro.local,aw-watcher-window_Ores-MacBook-Pro.local,2022-03-12T21:46:33.216551+00:00,None,currentwindow,aw-watcher-window,Ores-MacBook-Pro.local,[{'timestamp': '2022-03-17T18:18:54.373000+00:...
aw-watcher-window_SAW-L5487407OT9.hitronhub.home,aw-watcher-window_SAW-L5487407OT9.hitronhub.home,2022-03-08T23:10:08.239293+00:00,None,currentwindow,aw-watcher-window,SAW-L5487407OT9.hitronhub.home,[{'timestamp': '2022-03-13T05:44:14.760000+00:...


In [53]:
# get events from active windows

activity_watch_current_window = activity_watch[activity_watch['type'] == 'currentwindow']
# activity_watch_current_window = pd.DataFrame(activity_watch_current_window['events'])
activity_watch_current_window

,id,created,name,type,client,hostname,events
aw-watcher-window_MacBook-Pro.hitronhub.home,aw-watcher-window_MacBook-Pro.hitronhub.home,2022-01-04T21:15:37.737954+00:00,None,currentwindow,aw-watcher-window,MacBook-Pro.hitronhub.home,[{'timestamp': '2022-01-21T02:57:33.262000+00:...
aw-watcher-window_Ores-MBP.hitronhub.home,aw-watcher-window_Ores-MBP.hitronhub.home,2022-01-20T18:59:44.736005+00:00,None,currentwindow,aw-watcher-window,Ores-MBP.hitronhub.home,[{'timestamp': '2022-03-09T07:07:15.992000+00:...
aw-watcher-window_Ores-MacBook-Pro.local,aw-watcher-window_Ores-MacBook-Pro.local,2022-03-12T21:46:33.216551+00:00,None,currentwindow,aw-watcher-window,Ores-MacBook-Pro.local,[{'timestamp': '2022-03-17T18:18:54.373000+00:...
aw-watcher-window_SAW-L5487407OT9.hitronhub.home,aw-watcher-window_SAW-L5487407OT9.hitronhub.home,2022-03-08T23:10:08.239293+00:00,None,currentwindow,aw-watcher-window,SAW-L5487407OT9.hitronhub.home,[{'timestamp': '2022-03-13T05:44:14.760000+00:...
aw-watcher-window_og-mbpro,aw-watcher-window_og-mbpro,2022-03-17T11:26:13.229656+00:00,None,currentwindow,aw-watcher-window,og-mbpro,[{'timestamp': '2022-03-22T15:14:15.101000+00:...


In [54]:
# merge all the event data into one df

activity_watch_split = pd.DataFrame()
for row_index in range(len(activity_watch_current_window)):
    entry_df = pd.DataFrame(activity_watch_current_window['events'][row_index])
    activity_watch_split = pd.concat([activity_watch_split, entry_df], ignore_index=True)
    
activity_watch_split

,timestamp,duration,data
0,2022-01-21T02:57:33.262000+00:00,0.000,"{'app': 'Microsoft Edge', 'title': '*The 1995 ..."
1,2022-01-21T02:57:30.007000+00:00,2.162,"{'app': 'Microsoft Edge', 'title': 'Interact F..."
2,2022-01-21T02:57:27.842000+00:00,1.084,"{'app': 'Microsoft Edge', 'title': 'MY NLP JOU..."
3,2022-01-21T02:56:20.420000+00:00,66.340,"{'app': 'Microsoft Edge', 'title': 'MY NLP JOU..."
4,2022-01-21T02:56:18.202000+00:00,1.088,"{'app': 'Microsoft Edge', 'title': 'Neurosity ..."
...,...,...,...
62258,2022-03-17T18:26:26.271000+00:00,0.000,"{'app': 'Slack', 'title': 'Meta Careers - Micr..."
62259,2022-03-17T18:26:22.854000+00:00,2.300,"{'app': 'Slack', 'title': 'Slack | Nick Hanoia..."
62260,2022-03-17T18:26:20.602000+00:00,1.119,"{'app': 'Slack', 'title': 'Slack'}"
62261,2022-03-17T18:26:18.338000+00:00,1.139,"{'app': 'Mail', 'title': 'ML Engineering @Meta..."


In [55]:
#dropping the data column in a copy of this

activity_watch_split_timestamps = activity_watch_split.drop(columns=["data"])
activity_watch_split_timestamps

,timestamp,duration
0,2022-01-21T02:57:33.262000+00:00,0.000
1,2022-01-21T02:57:30.007000+00:00,2.162
2,2022-01-21T02:57:27.842000+00:00,1.084
3,2022-01-21T02:56:20.420000+00:00,66.340
4,2022-01-21T02:56:18.202000+00:00,1.088
...,...,...
62258,2022-03-17T18:26:26.271000+00:00,0.000
62259,2022-03-17T18:26:22.854000+00:00,2.300
62260,2022-03-17T18:26:20.602000+00:00,1.119
62261,2022-03-17T18:26:18.338000+00:00,1.139


In [56]:
#splitting the data column of activity_watch_split, to extract the app and title being used

activity_watch_split_events = activity_watch_split['data'].apply(pd.Series)
activity_watch_split_events

,app,title,url,incognito
0,Microsoft Edge,"*The 1995 SQL Reunion: People, Projects, and P...",NaN,NaN
1,Microsoft Edge,Interact Fellowship Application - Sleeping - M...,NaN,NaN
2,Microsoft Edge,MY NLP JOURNEY!!!!. GOAL: documenting my NLP j...,NaN,NaN
3,Microsoft Edge,MY NLP JOURNEY!!!!. GOAL: documenting my NLP j...,NaN,NaN
4,Microsoft Edge,Neurosity - Developer Console - Microsoft Edge,NaN,NaN
...,...,...,...,...
62258,Slack,Meta Careers - Microsoft Edge,NaN,NaN
62259,Slack,Slack | Nick Hanoian | Interact,NaN,NaN
62260,Slack,Slack,NaN,NaN
62261,Mail,ML Engineering @Meta (Facebook) | Canada 🇨🇦 — ...,NaN,NaN


In [57]:
#combining the two split activity watch dataframes so that it contains both the timestamp, app, title and duration

activity_watch_final = pd.concat([activity_watch_split_timestamps, activity_watch_split_events], axis=1)
activity_watch_final

,timestamp,duration,app,title,url,incognito
0,2022-01-21T02:57:33.262000+00:00,0.000,Microsoft Edge,"*The 1995 SQL Reunion: People, Projects, and P...",NaN,NaN
1,2022-01-21T02:57:30.007000+00:00,2.162,Microsoft Edge,Interact Fellowship Application - Sleeping - M...,NaN,NaN
2,2022-01-21T02:57:27.842000+00:00,1.084,Microsoft Edge,MY NLP JOURNEY!!!!. GOAL: documenting my NLP j...,NaN,NaN
3,2022-01-21T02:56:20.420000+00:00,66.340,Microsoft Edge,MY NLP JOURNEY!!!!. GOAL: documenting my NLP j...,NaN,NaN
4,2022-01-21T02:56:18.202000+00:00,1.088,Microsoft Edge,Neurosity - Developer Console - Microsoft Edge,NaN,NaN
...,...,...,...,...,...,...
62258,2022-03-17T18:26:26.271000+00:00,0.000,Slack,Meta Careers - Microsoft Edge,NaN,NaN
62259,2022-03-17T18:26:22.854000+00:00,2.300,Slack,Slack | Nick Hanoian | Interact,NaN,NaN
62260,2022-03-17T18:26:20.602000+00:00,1.119,Slack,Slack,NaN,NaN
62261,2022-03-17T18:26:18.338000+00:00,1.139,Mail,ML Engineering @Meta (Facebook) | Canada 🇨🇦 — ...,NaN,NaN


In [58]:
#checking for nulls in the title column that may not have been labelled as null

activity_watch_nulls = activity_watch_final[activity_watch_final['title'] == ""]
activity_watch_nulls

,timestamp,duration,app,title,url,incognito
14,2022-01-21T02:51:00.346000+00:00,258.953,FaceTime,,NaN,NaN
15,2022-01-21T02:50:59.170000+00:00,0.000,Messages,,NaN,NaN
51,2022-01-20T20:28:25.618000+00:00,0.000,Microsoft Edge,,NaN,NaN
229,2022-01-20T08:01:32.219000+00:00,0.000,Microsoft Edge,,NaN,NaN
249,2022-01-20T07:54:06.065000+00:00,0.000,Microsoft Edge,,NaN,NaN
...,...,...,...,...,...,...
61540,2022-03-18T20:57:52.087000+00:00,241.948,FaceTime,,NaN,NaN
61542,2022-03-18T20:55:54.803000+00:00,114.990,FaceTime,,NaN,NaN
61549,2022-03-18T20:51:54.834000+00:00,111.681,FaceTime,,NaN,NaN
61631,2022-03-18T20:21:34.866000+00:00,127.643,Tandem,,NaN,NaN


In [59]:
#extracting just the duplicates in the timestamp column

activity_watch_final_duplicates = activity_watch_final[activity_watch_final.duplicated(subset=['timestamp']) == True]
activity_watch_final_duplicates.duplicated(subset=['timestamp']).value_counts()

activity_watch_final_duplicates

#diving deeper into the duplicates to see if other variables such as "app" and "title" are the same
#if they are, it means that it is one app spawning multiple processes ( or one for each vscode window lol) - only for WindowsOS


#taking only the maximum value duplicate

activity_watch_final = activity_watch_final.sort_values('duration').drop_duplicates(subset=['timestamp','app','title'], keep='last')
activity_watch_final

,timestamp,duration,app,title,url,incognito
0,2022-01-21T02:57:33.262000+00:00,0.000,Microsoft Edge,"*The 1995 SQL Reunion: People, Projects, and P...",NaN,NaN
39588,2022-02-02T07:50:10.399000+00:00,0.000,FaceTime,,NaN,NaN
12547,2022-03-06T20:04:39.753000+00:00,0.000,Microsoft Edge,Twitter - Microsoft Edge,NaN,NaN
12548,2022-03-06T20:04:38.460000+00:00,0.000,Microsoft Edge,https://twitter.com/home - Microsoft Edge,NaN,NaN
12549,2022-03-06T20:04:37.150000+00:00,0.000,Microsoft Edge,Watch General Magic | Prime Video - Microsoft ...,NaN,NaN
...,...,...,...,...,...,...
3085,2022-01-15T09:32:30.049000+00:00,26120.828,Microsoft Edge,NaN,NaN,NaN
26679,2022-02-17T07:00:53.730000+00:00,31511.567,Spotify,NaN,NaN,NaN
13595,2022-03-05T06:25:28.748000+00:00,33661.596,Spotify,NaN,NaN,NaN
36146,2022-02-07T07:20:30.962000+00:00,43238.025,Messages,NaN,NaN,NaN


In [60]:
#renaming duration and timestamp column to reflect the appropriate units

activity_watch_final.rename(columns={"duration":"duration_seconds"}, inplace=True)
activity_watch_final.rename(columns={"timestamp":"timestamp_utc"}, inplace=True)
activity_watch_final

,timestamp_utc,duration_seconds,app,title,url,incognito
0,2022-01-21T02:57:33.262000+00:00,0.000,Microsoft Edge,"*The 1995 SQL Reunion: People, Projects, and P...",NaN,NaN
39588,2022-02-02T07:50:10.399000+00:00,0.000,FaceTime,,NaN,NaN
12547,2022-03-06T20:04:39.753000+00:00,0.000,Microsoft Edge,Twitter - Microsoft Edge,NaN,NaN
12548,2022-03-06T20:04:38.460000+00:00,0.000,Microsoft Edge,https://twitter.com/home - Microsoft Edge,NaN,NaN
12549,2022-03-06T20:04:37.150000+00:00,0.000,Microsoft Edge,Watch General Magic | Prime Video - Microsoft ...,NaN,NaN
...,...,...,...,...,...,...
3085,2022-01-15T09:32:30.049000+00:00,26120.828,Microsoft Edge,NaN,NaN,NaN
26679,2022-02-17T07:00:53.730000+00:00,31511.567,Spotify,NaN,NaN,NaN
13595,2022-03-05T06:25:28.748000+00:00,33661.596,Spotify,NaN,NaN,NaN
36146,2022-02-07T07:20:30.962000+00:00,43238.025,Messages,NaN,NaN,NaN


In [61]:
#checking for numerical anomalies that look too small

# activity_watch_final[activity_watch_final["duration_seconds"] == 0]

### TODO: Write to out to it's own dataset & Create a category for tabs open but never visited

#checking for numerical anomalies that look too big

# activity_watch_final[activity_watch_final["duration_seconds"] > 14].sort_values("duration_seconds", ascending=False)[30:50]

In [62]:
#removing rows with LockApp

activity_watch_final.drop(index=activity_watch_final[activity_watch_final['app'] == 'LockApp.exe'].index, inplace=True)

#removing rows with unknown

activity_watch_final.drop(index=activity_watch_final[activity_watch_final['app'] == 'unknown'].index, inplace=True)

In [63]:
#checking for different variations of the same name
activity_watch_final["app"].value_counts()

#sorting the values from the earliest date to the latest date
activity_watch_final.sort_values("timestamp_utc", ascending=True, inplace=True)

#resetting the index values
activity_watch_final.reset_index(drop=True, inplace=True)
activity_watch_final

,timestamp_utc,duration_seconds,app,title,url,incognito
0,2022-01-05T05:16:06.837000+00:00,4.466,System Preferences,Security & Privacy,NaN,NaN
1,2022-01-05T05:16:12.430000+00:00,3.342,Finder,Applications,NaN,NaN
2,2022-01-05T05:16:16.898000+00:00,56.149,Microsoft Edge,ActivityWatch - Microsoft Edge,NaN,NaN
3,2022-01-05T05:17:14.172000+00:00,4.511,Microsoft Edge,macOS: add support for macOS 11 and Apple M1 S...,NaN,NaN
4,2022-01-05T05:17:19.789000+00:00,16.856,Microsoft Edge,MacOS Catalina - unable to start app · Issue #...,NaN,NaN
...,...,...,...,...,...,...
61061,2022-03-22T15:12:00.657000+00:00,7.628,Microsoft Edge,Dash - Microsoft Edge,NaN,NaN
61062,2022-03-22T15:12:09.383000+00:00,83.719,Microsoft Edge,activity_watch_processing - Jupyter Notebook -...,NaN,NaN
61063,2022-03-22T15:13:34.213000+00:00,36.514,Messages,Adaobi - hypeman 👾,NaN,NaN
61064,2022-03-22T15:14:11.834000+00:00,2.165,Code,main.py — Question (Workspace),NaN,NaN


Rename activity titles

In [64]:
#For names that include "code" we will be changing it to "VSCode" to avoid double counting
#For names that include "spotify" we will be changing it to "Spotify" to avoid double counting

def preprocess_app_name(value):
    if value == "Code":
        return "VSCode"
    if value == "spotify":
        return "Spotify"
    return value
        
activity_watch_final['app'] = activity_watch_final["app"].apply(lambda x: preprocess_app_name(x))

#For titles that are empty we are converting them to null values

activity_watch_final["title"] = activity_watch_final[["title", 'app']].apply(
                                    lambda x: x['app'] if not x['title'] else x['title'], axis = 1)


# set(activity_watch_final['app'].tolist())

## extracting topics using yake

In [65]:
language = "en"
max_ngram_size = 4
deduplication_threshold = 0.9
numOfKeywords = 5 # the max number of results to expect

#creating a function that removes key words with low value, and then extracting key words for titles with "msedge" as the named app
browsing_yake_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)

activity_watch_edge_events = activity_watch_final[ (activity_watch_final["app"] == "msedge.exe") | (activity_watch_final["app"] == "Microsoft Edge")]

skip_words = ["Personal", "Microsoft", "Edge", "pages", "tab", "slack", "Ọrẹ̀", "Ore", "oreHGA", "April", "Ogundipe", "Timi", "ORE_OGUNDIPE_T4.pdf", "oreHGA", "(Ore Ogundipe)", "Ọrẹ̀ Ògúndípẹ̀", "API" ]

def getkeywords(text):
    #TODO: get rid of stop words before doing extraction
    # - punctuations
    # - "Personal - Microsoft Edge"
    if type(text) is not str:
        return None

    text = text.replace(" - Personal - Microsoft Edge", "")
    text = text.replace("slack", "")
    text = text.replace("Slack", "")
    text = text.replace("Twitter", "")
    text = text.replace("tweets", "")
    text = text.replace("Ore", "")
    text = text.replace("Ọrẹ̀", "")
    text = text.replace("Orehga", "")
    text = text.replace("oreHGA", "")
    text = text.replace("April", "")
    text = text.replace("Ogundipe", "")
    text = text.replace("Timi", "")
    text = text.replace("ORE_OGUNDIPE_T4.pdf", "")
    text = text.replace("(Ore Ogundipe)", "")
    text = text.replace("Ọrẹ̀ Ògúndípẹ̀", "")
    text = text.replace("buraksekili/", "")
    text = text.replace("meet", "")
    text = text.replace("Meet", "")
    text = text.replace("- Microsoft Edge", "")
    text = text.replace("Microsoft Edge", "")

    yake_keywords = browsing_yake_extractor.extract_keywords(text)

    resulting_keywords = []
    for result in yake_keywords:
        if result[0] in skip_words:
            continue
        # convert all keyword entries to lower case
        # each keyword has a structure of (keyword, confidence)
        resulting_keywords.append((result[0].lower(), result[1]))
    
    return resulting_keywords

activity_watch_edge_events["keywords"] = activity_watch_edge_events["title"].apply(lambda x: getkeywords(x))
activity_watch_edge_events["keywords"]



<ipython-input-65-f5df3b38b072>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_edge_events["keywords"] = activity_watch_edge_events["title"].apply(lambda x: getkeywords(x))


2                   [(activitywatch, 0.04491197687864554)]
3        [(pull request, 0.004007038947002766), (add su...
4        [(unable to start app, 0.0015043378032723917),...
8        [(unable to start app, 0.0015043378032723917),...
9                   [(activitywatch, 0.04491197687864554)]
                               ...                        
61055    [(jupyter notebook, 0.007444681613352736), (ac...
61058    [(jupyter notebook, 0.007444681613352736), (ac...
61060    [(jupyter notebook, 0.007444681613352736), (ac...
61061                        [(dash, 0.04491197687864554)]
61062    [(jupyter notebook, 0.007444681613352736), (ac...
Name: keywords, Length: 32013, dtype: object

we then go ahead to a frequency mapping for the event classifications to pick the most relevant

In [66]:
browser_keyword_freq_dict = {}
for keyword_entries in activity_watch_edge_events["keywords"]:
    if keyword_entries is None:
        continue
    
    # each keyword has a structure of (keyword, confidence)
    # we don't need confidence so we only take keyword
    for single_keyword_entry in keyword_entries:
        keyword = single_keyword_entry[0]
        if browser_keyword_freq_dict.get(keyword) == None:
            browser_keyword_freq_dict[keyword] = 1
        else:
            browser_keyword_freq_dict[keyword] += 1

# the result of browser_keyword_freq_dict has a strcuture of (keyword, appearance_count)
sorted_browser_keyword_freq_dict = dict(sorted(browser_keyword_freq_dict.items(), key=lambda item: item[1], reverse=True))



the result of browser_keyword_freq_dict has a strcuture of (keyword, appearance_count)

now we select a single winner called "context" from the keyword entries

In [67]:
#choosing the most relevant topic out of all the selected topics

def choose_winning_key(keyword_entries):
    """This is chosen as a function of occurence in total data set
    and confidence
    
    use weight ( confidence * keyword frequency in total dataset) to
    decide what is the winning keyword/event classification"""
    if keyword_entries is None or len(keyword_entries) < 1:
        return None
        
    keyword_weights_dict = {}
    # check frequency count for each word in descending order of confidence
    for entry in keyword_entries:
        # essentially weight = confidence * frequency
        keyword_weights_dict[entry[0]] = entry[1] * browser_keyword_freq_dict[entry[0]]

    # the dictionary will be of structure {"event_keyword", "weight"}
    # pick the keyword with the highest weight
    sorted_keyword_weights_dict = dict(sorted(keyword_weights_dict.items(), key=lambda item: item[1], reverse=True))
    
    #TODO:what happens when there's a tie - for now we're just picking the heighest
    return list(sorted_keyword_weights_dict.keys())[0]
    


activity_watch_edge_events["event_classification"] = activity_watch_edge_events["keywords"].apply(choose_winning_key)

activity_watch_edge_events["event_classification"].value_counts()

<ipython-input-67-c1dc20873827>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_edge_events["event_classification"] = activity_watch_edge_events["keywords"].apply(choose_winning_key)


latest             1562
question           1530
search             1295
tab                1038
google              592
                   ... 
ethereum              1
btw                   1
mhope                 1
resembling data       1
days                  1
Name: event_classification, Length: 1762, dtype: int64

## Get an event classification for VSCode events

In [68]:
#Removing words from VSCode that gives us little information, and classifying based on hihg value words

activity_watch_events_vscode = activity_watch_final[activity_watch_final["app"] == "VSCode"]
activity_watch_events_vscode

def get_vscode_context(text):
    if type(text) is not str:
        return (None, None)
    # output from this will be "filename", "project"
    # first split string with space " - "
    text = text.replace("Visual Studio Code - Insiders", "")
    text = text.replace("Visual Studio Code", "")
    text = text.replace("(Untracked)", "")
    text = text.replace("(Working Tree)", "")
    text = text.replace("●", "")
    text = text.rstrip(" - ")

    # VSCode has a structure of how the window title is defined (need to validate if this is same on mac)
    # filename - projectname - Visual Studio Code
    # on windows
    text_split = text.split(" - ")
    if len(text_split) < 2: #ensure it's been broken into components
        # on mac
        text_split = text.split(" — ")
    # sort in descending order
    reverse_text_split_sort = sorted(text_split, key=text_split.index, reverse=True)

    # item 0 will be project name
    # item 1 will be the file name
    project = None
    filename = None

    if len(reverse_text_split_sort) == 1:
        project = reverse_text_split_sort[0]

    if len(reverse_text_split_sort) >= 2:
        project = reverse_text_split_sort[0]
        filename = reverse_text_split_sort[1]

    # return a tuple "(project, filename)"
    return (project, filename)


# calling the column keywords to be consistent with activity watch
activity_watch_events_vscode["keywords"] = activity_watch_events_vscode["title"].apply(get_vscode_context)

#seeing what the data looks like
activity_watch_events_vscode["event_classification"] = activity_watch_events_vscode["keywords"].apply(lambda x: x[0])

# activity_watch_events_vscode

combined_aw_edge_vscode =  activity_watch_edge_events.append(activity_watch_events_vscode)
combined_aw_edge_vscode_final = combined_aw_edge_vscode.drop(["timestamp_utc", "app", "duration_seconds", "title"], axis=1)
combined_aw_edge_vscode_final

<ipython-input-68-30f12db70828>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_events_vscode["keywords"] = activity_watch_events_vscode["title"].apply(get_vscode_context)
<ipython-input-68-30f12db70828>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activity_watch_events_vscode["event_classification"] = activity_watch_events_vscode["keywords"].apply(lambda x: x[0])
<ipython-input-68-30f12db70828>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

,url,incognito,keywords,event_classification
2,NaN,NaN,"[(activitywatch, 0.04491197687864554)]",activitywatch
3,NaN,NaN,"[(pull request, 0.004007038947002766), (add su...",pull request
4,NaN,NaN,"[(unable to start app, 0.0015043378032723917),...",issue
8,NaN,NaN,"[(unable to start app, 0.0015043378032723917),...",issue
9,NaN,NaN,"[(activitywatch, 0.04491197687864554)]",activitywatch
...,...,...,...,...
61044,NaN,NaN,"(Question (Workspace), main.py)",Question (Workspace)
61046,NaN,NaN,"(Question (Workspace), main.py)",Question (Workspace)
61052,NaN,NaN,"(Question (Workspace), main.py)",Question (Workspace)
61054,NaN,NaN,"(Question (Workspace), main.py)",Question (Workspace)


## combine vscode & msedge events with other app activities

In [69]:

activity_watch_dataframe_final = pd.concat([activity_watch_final, combined_aw_edge_vscode_final], axis=1, join="outer")
activity_watch_dataframe_final

,timestamp_utc,duration_seconds,app,title,url,incognito,url,incognito,keywords,event_classification
0,2022-01-05T05:16:06.837000+00:00,4.466,System Preferences,Security & Privacy,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-05T05:16:12.430000+00:00,3.342,Finder,Applications,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-05T05:16:16.898000+00:00,56.149,Microsoft Edge,ActivityWatch - Microsoft Edge,NaN,NaN,NaN,NaN,"[(activitywatch, 0.04491197687864554)]",activitywatch
3,2022-01-05T05:17:14.172000+00:00,4.511,Microsoft Edge,macOS: add support for macOS 11 and Apple M1 S...,NaN,NaN,NaN,NaN,"[(pull request, 0.004007038947002766), (add su...",pull request
4,2022-01-05T05:17:19.789000+00:00,16.856,Microsoft Edge,MacOS Catalina - unable to start app · Issue #...,NaN,NaN,NaN,NaN,"[(unable to start app, 0.0015043378032723917),...",issue
...,...,...,...,...,...,...,...,...,...,...
61061,2022-03-22T15:12:00.657000+00:00,7.628,Microsoft Edge,Dash - Microsoft Edge,NaN,NaN,NaN,NaN,"[(dash, 0.04491197687864554)]",dash
61062,2022-03-22T15:12:09.383000+00:00,83.719,Microsoft Edge,activity_watch_processing - Jupyter Notebook -...,NaN,NaN,NaN,NaN,"[(jupyter notebook, 0.007444681613352736), (ac...",jupyter
61063,2022-03-22T15:13:34.213000+00:00,36.514,Messages,Adaobi - hypeman 👾,NaN,NaN,NaN,NaN,NaN,NaN
61064,2022-03-22T15:14:11.834000+00:00,2.165,VSCode,main.py — Question (Workspace),NaN,NaN,NaN,NaN,"(Question (Workspace), main.py)",Question (Workspace)


In [70]:
import math

def update_classification_state(entry):
    """
    if dataframe does not have an event_classification,
    check if title exists then update,
    check for app and then update
    """
    if pd.isna(entry['event_classification']) or entry['event_classification'] == "":
        if pd.isna(entry['title']) or entry['title'] == "":
            return entry['app']
        return entry['title']
    return entry['event_classification']



activity_watch_dataframe_final['event_classification'] = activity_watch_dataframe_final[['app','title','event_classification']].apply(
    lambda x: update_classification_state(x), axis=1
)
activity_watch_dataframe_final

,timestamp_utc,duration_seconds,app,title,url,incognito,url,incognito,keywords,event_classification
0,2022-01-05T05:16:06.837000+00:00,4.466,System Preferences,Security & Privacy,NaN,NaN,NaN,NaN,NaN,Security & Privacy
1,2022-01-05T05:16:12.430000+00:00,3.342,Finder,Applications,NaN,NaN,NaN,NaN,NaN,Applications
2,2022-01-05T05:16:16.898000+00:00,56.149,Microsoft Edge,ActivityWatch - Microsoft Edge,NaN,NaN,NaN,NaN,"[(activitywatch, 0.04491197687864554)]",activitywatch
3,2022-01-05T05:17:14.172000+00:00,4.511,Microsoft Edge,macOS: add support for macOS 11 and Apple M1 S...,NaN,NaN,NaN,NaN,"[(pull request, 0.004007038947002766), (add su...",pull request
4,2022-01-05T05:17:19.789000+00:00,16.856,Microsoft Edge,MacOS Catalina - unable to start app · Issue #...,NaN,NaN,NaN,NaN,"[(unable to start app, 0.0015043378032723917),...",issue
...,...,...,...,...,...,...,...,...,...,...
61061,2022-03-22T15:12:00.657000+00:00,7.628,Microsoft Edge,Dash - Microsoft Edge,NaN,NaN,NaN,NaN,"[(dash, 0.04491197687864554)]",dash
61062,2022-03-22T15:12:09.383000+00:00,83.719,Microsoft Edge,activity_watch_processing - Jupyter Notebook -...,NaN,NaN,NaN,NaN,"[(jupyter notebook, 0.007444681613352736), (ac...",jupyter
61063,2022-03-22T15:13:34.213000+00:00,36.514,Messages,Adaobi - hypeman 👾,NaN,NaN,NaN,NaN,NaN,Adaobi - hypeman 👾
61064,2022-03-22T15:14:11.834000+00:00,2.165,VSCode,main.py — Question (Workspace),NaN,NaN,NaN,NaN,"(Question (Workspace), main.py)",Question (Workspace)


In [71]:
#changing mislaballed topics

def format_label(label):
    if type(label) == str:
        label = label.lower()

    if label == "eeg-notebooks developers":
        label = "eeg-notebooks"
    if label == "calendly":
        label = "planning"
    if label == "google calendar":
        label = "planning"
    if label == "latest tweets":
        label = "tweets"
    if label == "brain-dump":
        label = "braindump"
    if label == "eeg-notebooks project":
        label = "eeg-notebooks"
    if label == "spotify streaming":
        label = "spotify"
    if label == "openai-api":
        label = "openai api"
        
    
    return label

activity_watch_dataframe_final["event_classification"] = activity_watch_dataframe_final["event_classification"].apply(format_label)

format and explort activity watch data

In [72]:
activity_watch_dataframe_final

,timestamp_utc,duration_seconds,app,title,url,incognito,url,incognito,keywords,event_classification
0,2022-01-05T05:16:06.837000+00:00,4.466,System Preferences,Security & Privacy,NaN,NaN,NaN,NaN,NaN,security & privacy
1,2022-01-05T05:16:12.430000+00:00,3.342,Finder,Applications,NaN,NaN,NaN,NaN,NaN,applications
2,2022-01-05T05:16:16.898000+00:00,56.149,Microsoft Edge,ActivityWatch - Microsoft Edge,NaN,NaN,NaN,NaN,"[(activitywatch, 0.04491197687864554)]",activitywatch
3,2022-01-05T05:17:14.172000+00:00,4.511,Microsoft Edge,macOS: add support for macOS 11 and Apple M1 S...,NaN,NaN,NaN,NaN,"[(pull request, 0.004007038947002766), (add su...",pull request
4,2022-01-05T05:17:19.789000+00:00,16.856,Microsoft Edge,MacOS Catalina - unable to start app · Issue #...,NaN,NaN,NaN,NaN,"[(unable to start app, 0.0015043378032723917),...",issue
...,...,...,...,...,...,...,...,...,...,...
61061,2022-03-22T15:12:00.657000+00:00,7.628,Microsoft Edge,Dash - Microsoft Edge,NaN,NaN,NaN,NaN,"[(dash, 0.04491197687864554)]",dash
61062,2022-03-22T15:12:09.383000+00:00,83.719,Microsoft Edge,activity_watch_processing - Jupyter Notebook -...,NaN,NaN,NaN,NaN,"[(jupyter notebook, 0.007444681613352736), (ac...",jupyter
61063,2022-03-22T15:13:34.213000+00:00,36.514,Messages,Adaobi - hypeman 👾,NaN,NaN,NaN,NaN,NaN,adaobi - hypeman 👾
61064,2022-03-22T15:14:11.834000+00:00,2.165,VSCode,main.py — Question (Workspace),NaN,NaN,NaN,NaN,"(Question (Workspace), main.py)",question (workspace)


In [73]:
from datetime import datetime, timedelta
# --- initial potential 
#activity_watch_summarized = summary_day, total_time_surfing, what_app_did_user_spend_x_percent_of_time_on (one column per percent) , events_usage (tuple: event_classification (encoded), sum_total_time)
# i realised that this can actually be better derived on the fly with visualizations
# when summarize this much, we lose a lot of granularity

activity_watch_dataframe_final["summary_day"] = activity_watch_dataframe_final['timestamp_utc'].apply(
    lambda timestamp: datetime.fromisoformat(timestamp).strftime("%Y-%m-%d"))

N=10
aw_summary_grouped = activity_watch_dataframe_final.groupby(["summary_day", "app", "event_classification"], as_index=False)
aw_summary_durations = aw_summary_grouped.agg({
                            'duration_seconds': 'sum'
                        })
# aw_summary_durations = aw_summary_durations.apply(lambda g: g.sort_values(by=["duration_seconds"], ascending=False).head(N))
aw_summary_durations['summary_day_end'] = aw_summary_durations['summary_day'].apply(
    lambda value: (datetime.strptime(value, "%Y-%m-%d") + timedelta(days=1))
)


aw_summary_durations.to_csv(f'../data/aw_data_grouped_{datetime.now().strftime("%Y_%m_%d")}.csv', index=False)

aw_summary_durations



,summary_day,app,event_classification,duration_seconds,summary_day_end
0,2022-01-05,Finder,applications,4.468,2022-01-06
1,2022-01-05,Google Chrome,bell media account,4.259,2022-01-06
2,2022-01-05,Google Chrome,"crave | watch hbo, showtime and starz movies a...",92.820,2022-01-06
3,2022-01-05,Google Chrome,daria,3.214,2022-01-06
4,2022-01-05,Google Chrome,"daria | s4:e4 | murder, she snored",3967.816,2022-01-06
...,...,...,...,...,...
9970,2022-03-22,VSCode,question (workspace),497.810,2022-03-23
9971,2022-03-22,VSCode,vscode,1340.110,2022-03-23
9972,2022-03-22,Warp,warp,40.783,2022-03-23
9973,2022-03-22,pgAdmin 4,pgadmin 4,3.275,2022-03-23


In [74]:
aw_summary_durations = aw_summary_durations.groupby(["app"], as_index=False).agg({'duration_seconds': sum}).sort_values(by=['duration_seconds'], ascending=False)
aw_summary_durations['duration_minutes'] = aw_summary_durations['duration_seconds'].apply( lambda value: value // 60 )

aw_summary_durations['duration_hours'] = aw_summary_durations['duration_seconds'].apply( lambda value: round(value / (60*60), 1))


aw_summary_durations

,app,duration_seconds,duration_minutes,duration_hours
19,Microsoft Edge,1109711.646,18495.0,308.3
17,Messages,251294.813,4188.0,69.8
36,Spotify,205458.559,3424.0,57.1
45,VSCode,154948.283,2582.0,43.0
26,Notion,133642.695,2227.0,37.1
40,Tandem,119265.171,1987.0,33.1
8,FaceTime,54442.297,907.0,15.1
16,Mail,39706.526,661.0,11.0
46,Warp,34722.775,578.0,9.6
52,zoom.us,27771.313,462.0,7.7
